In [ ]:
!pip install python-crfsuite
from itertools import chain
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
import pycrfsuite
import numpy as np
import pandas as pd


ne_path ="/content/drive/MyDrive/Colab Notebooks/project4_data/NE.txt"
ma_path = "/content/drive/MyDrive/Colab Notebooks/project4_data/NE.ma.txt"

     |████████████████████████████████| 747kB 9.4MB/s 


In [ ]:
my_data = []
with open(ne_path) as f:
    for line in f:
      my_data.append(line[:-1])

In [ ]:
# ma_path = "NE.ma.txt"
# ne_path = "NE.txt"

my_data = []
with open(ne_path) as f:
    for line in f:
      my_data.append(line[:-1])

tagged_data = []
mode = 0 # 0->no tag, 1->beg_tag 2-> mid_tag
for i in range(len(my_data)):
  for each in my_data[i].split():
    if mode == 0: # no tag
      if each == "<b_enamex":
        mode = 1
      else:
        tagged_data.append(str(i+1) + " " + each + " O")
    elif mode == 1: # beg_tag
      tag_type = each.split("\"")[1]
      if "<e_enamex>" in each:
        tagged_data.append(str(i+1) + " " + each.split("\"")[2].split("<")[0][1:] + " B-" + tag_type[:3] )
        mode = 0
      else:
        tagged_data.append(str(i+1) + " " + each.split("\"")[2].split(">")[1] + " B-" + tag_type[:3] )
        mode = 2
    elif mode == 2: # mid_tag
      if "<e_enamex>" in each:
        tagged_data.append(str(i+1) + " " + each.split("<")[0] + " I-" + tag_type[:3] )
        mode = 0
      else:
        tagged_data.append(str(i+1) + " " + each + " I-" + tag_type[:3] )


ner_tagged_data = tagged_data
ner_tagged_list_of_sentences = []

line_cnt = 1
temp_line = []
for each in ner_tagged_data:
  current_line = int(each.split()[0])
  if current_line == line_cnt:
    temp_line.append(each)
  else:
    ner_tagged_list_of_sentences.append(temp_line)
    temp_line = [each]
    line_cnt += 1
ner_tagged_list_of_sentences.append(temp_line)


ma_data = []
with open(ma_path) as f:
    for line in f:
        ma_data.append(line[:-1])
ma_data_list_of_sentences = []

line_cnt = 1
temp_line = []
for each in ma_data:
  current_line = int(each.split()[0])
  if current_line == line_cnt:
    temp_line.append(each)
  else:
    ma_data_list_of_sentences.append(temp_line)
    temp_line = [each]
    line_cnt += 1
ma_data_list_of_sentences.append(temp_line)


# for i in range(len(ma_data_list_of_sentences)):
#     sentence = ma_data_list_of_sentences[i]
#     for j in range(len(sentence)):
#         word = sentence[j]
#         ner_cnt = j
#         ner_tag = "O"
#         while ner_cnt < len(ner_tagged_list_of_sentences[i]):
#             if ner_tagged_list_of_sentences[i][ner_cnt].split()[1] in word:
#                 ner_tag = ner_tagged_list_of_sentences[i][ner_cnt].split()[2]
#                 break
#             else:
#                 ner_cnt += 1
#         word += " " + ner_tag
#         ma_data_list_of_sentences[i][j] = word

df = []

for i in range(len(ner_tagged_list_of_sentences)):
    sentence = ner_tagged_list_of_sentences[i]
    ma_cnt = 0
    temp_sent = []
    for j in range(len(sentence)):
      word = sentence[j]
      if ma_cnt < len(ma_data_list_of_sentences[i])  and  word.split()[1] in ma_data_list_of_sentences[i][ma_cnt].split()[1]:
        ma_tag = ma_data_list_of_sentences[i][ma_cnt].split()[2]
        ma_cnt += 1
      else:
        ma_tag = "*UNKNOWN*"
      line_no = word.split()[0]
      token = word.split()[1]
      if(len(word.split()) < 3):
        ner_tag = word.split()[1]
      else:
        ner_tag = word.split()[2]

      temp_sent.append(line_no + " " + token + " " + ma_tag + " " + ner_tag)
    df.append(temp_sent)
  




ner_data_sentenced = df

In [ ]:
df[0]

['1 Müzik müzik+Noun+A3sg+Pnon+Nom O',
 '1 Şenliği Şenlik+Noun+Prop+A3sg+P3sg+Dat O',
 "1 'ne *UNKNOWN* O",
 '1 hazırlanın hazırla+Verb^DB+Verb+Reflex+Pos+Imp+A2pl O',
 '1 POZİTİF pozitif+Adj B-ORG',
 '1 ve ve+Conj I-ORG',
 '1 Açık açık+Adj I-ORG',
 '1 Radyo radyo+Noun+A3sg+Pnon+Nom I-ORG',
 '1 işbirliğiyle işbirliği+Noun+A3sg+Pnon+Ins O',
 '1 düzenlenecek düzenle+Verb^DB+Verb+Pass+Pos^DB+Adj+FutPart+Pnon O',
 '1 olan ol+Verb+Pos^DB+Adj+PresPart O',
 '1 İstanbul İstanbul+Noun+Prop+A3sg+Pnon+Nom B-LOC',
 '1 Müzik müzik+Noun+A3sg+Pnon+Nom O',
 '1 Şenliği şenlik+Noun+A3sg+P3sg+Nom O',
 '1 2 2+Num+Card O',
 '1 , ,+Punct O',
 '1 müzikseverlere müziksever+Noun+A3pl+Pnon+Dat O',
 '1 Aralık aralık+Noun+A3sg+Pnon+Nom O',
 '1 ayında ay+Noun+A3sg+P3sg+Loc O',
 '1 merhaba merhaba+Noun+A3sg+Pnon+Nom O',
 '1 demeye de+Verb+Pos^DB+Noun+Inf2+A3sg+Pnon+Dat O',
 '1 hazırlanıyor hazırla+Verb^DB+Verb+Pass+Pos+Prog1+A3sg O']

In [ ]:
to_file = []
for each in ner_data_sentenced:
  for a in each:
    to_file.append(a+"\n")
with open("/content/drive/MyDrive/Colab Notebooks/project4_data/NE.prep.txt", "w") as file1:
    file1.writelines(to_file)

In [ ]:
tagged_data[0]

'1 Müzik O'